# Importing functions

In [2]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
import re
import random 
import numpy as np
import warnings
import tensorflow as tf 
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import make_column_transformer
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from preprocess_days_stats import preprocess_match_days
from preprocess_time_serie import preprocess_teams, create_time_series_features
from preprocess_time_series_features import preprocess_features_time_series, create_fast_preprocessing_ts, preprocess_features_time_series_odds, create_fast_preprocessing_ts_odds
from create_models_time_series import create_time_series_model_dense, create_time_series_model_lstm, create_time_series_model_conv1d, create_time_series_model_odds
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams, plot_loss_curve

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

# Adding the new results to the dataframe

In [14]:
# Leggi il file CSV
df = pd.read_csv(r"C:\Users\Hp\Documents\Serie_A_dump\csv_predictions\december_four.csv", parse_dates=['Date'], dayfirst=True)
df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')

# Crea un DataFrame con 10 righe di zeri
new_rows = pd.DataFrame(0, index=range(10), columns=new_csv.columns)

# Assegna valori specifici alle colonne 'Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam' per ciascuna riga
new_rows.loc[0, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Juventus', 'Napoli']
new_rows.loc[1, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Verona', 'Lazio']
new_rows.loc[2, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Atalanta', 'Milan']
new_rows.loc[3, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Inter', 'Udinese']
new_rows.loc[4, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Frosinone', 'Torino']
new_rows.loc[5, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Monza', 'Genoa']
new_rows.loc[6, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Salernitana', 'Bologna']
new_rows.loc[7, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Roma', 'Fiorentina']
new_rows.loc[8, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Empoli', 'Lecce']
new_rows.loc[9, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', '10/12/2023', '17:30', 'Cagliari', 'Sassuolo']


# Aggiungi le nuove righe al DataFrame esistente
new_csv = pd.concat([df, new_rows], ignore_index=True)

# Salva il DataFrame aggiornato su un nuovo file CSV
new_csv.to_csv(r"C:\Users\Hp\Serie_A\csv_predictions\november_four.csv")

In [15]:
df.sort_values(['Date'], ascending=False).head(55)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
99,I1,30/10/2023,19:45,Lazio,Fiorentina,1,0,H,0,0,D,11,9,4,5,12,17,0,4,3,3,0,0,2.25,3.25,3.25,2.25,3.40,3.10,2.30,3.35,3.15,2.31,3.48,3.24,2.30,3.40,3.00,2.20,3.10,3.13,2.38,3.60,3.34,2.30,3.42,3.17,1.99,1.91,1.98,1.90,2.03,1.96,1.96,1.88,-0.25,2.00,1.93,1.99,1.92,2.00,1.94,1.97,1.88,2.25,3.30,3.25,2.35,3.25,3.10,2.30,3.30,3.15,2.33,3.38,3.30,2.30,3.30,3.10,2.25,3.20,3.25,2.42,3.50,3.61,2.32,3.35,3.21,2.02,1.88,2.01,1.90,2.06,1.99,1.96,1.87,-0.25,1.99,1.94,2.01,1.92,2.04,1.97,1.97,1.88
98,I1,30/10/2023,17:30,Empoli,Atalanta,0,3,A,0,2,A,9,18,2,6,10,13,3,6,4,0,0,0,5.25,3.75,1.67,4.80,3.90,1.68,4.90,3.90,1.70,5.03,4.08,1.70,4.80,3.90,1.67,5.00,3.50,1.62,5.25,4.10,1.77,4.92,3.96,1.70,1.75,2.05,1.83,2.06,1.85,2.11,1.79,2.05,0.75,2.02,1.91,2.00,1.90,2.02,1.91,1.96,1.87,5.00,4.00,1.65,4.75,4.00,1.68,5.50,4.20,1.60,5.06,4.15,1.67,5.00,4.00,1.62,5.50,3.90,1.62,5.50,4.37,1.70,5.09,4.09,1.65,1.70,2.10,1.75,2.20,1.75,2.23,1.71,2.14,0.75,2.04,1.89,2.05,1.88,2.12,1.90,2.03,1.81
62,I1,30/09/2023,19:45,Salernitana,Inter,0,4,A,0,0,D,12,16,3,6,12,10,3,3,2,1,0,0,8.00,4.75,1.40,7.25,4.75,1.42,7.50,4.80,1.40,8.16,4.81,1.43,7.50,4.75,1.40,8.00,4.80,1.36,8.30,5.00,1.46,7.79,4.78,1.42,1.67,2.15,1.72,2.19,1.75,2.25,1.70,2.19,1.25,1.97,1.93,1.97,1.93,2.00,1.97,1.93,1.91,8.50,4.75,1.40,7.50,4.75,1.41,7.50,4.60,1.43,7.89,4.96,1.43,8.00,5.00,1.36,8.00,5.00,1.36,9.34,5.20,1.46,8.00,4.88,1.41,1.67,2.15,1.71,2.25,1.72,2.33,1.69,2.20,1.25,2.00,1.93,1.98,1.94,2.05,1.95,1.97,1.88
61,I1,30/09/2023,17:00,Milan,Lazio,2,0,H,0,0,D,14,13,7,4,15,5,7,1,3,2,0,0,1.73,3.60,5.00,1.75,3.70,4.60,1.75,3.60,5.00,1.78,3.72,5.04,1.73,3.70,4.75,1.73,3.40,5.25,1.82,3.87,5.25,1.77,3.70,4.84,2.06,1.84,2.06,1.84,2.07,1.88,2.00,1.83,-0.75,2.01,1.89,2.01,1.90,2.04,1.91,1.98,1.86,1.73,3.60,5.00,1.73,3.70,4.80,1.70,3.70,5.25,1.80,3.76,4.95,1.67,3.75,5.50,1.67,3.60,5.50,1.82,3.90,5.66,1.74,3.75,5.06,2.00,1.90,1.99,1.92,2.02,1.95,1.94,1.89,-0.75,2.02,1.91,2.04,1.89,2.05,2.01,1.93,1.91
60,I1,30/09/2023,14:00,Lecce,Napoli,0,4,A,0,1,A,9,15,4,6,11,11,3,7,3,2,0,0,4.75,3.80,1.75,4.40,3.70,1.78,4.60,3.75,1.75,4.84,3.90,1.76,4.80,3.75,1.70,4.80,3.70,1.73,5.03,3.95,1.80,4.73,3.80,1.76,2.01,1.89,2.00,1.88,2.02,1.94,1.95,1.87,0.75,1.92,1.98,1.93,1.98,1.93,2.00,1.88,1.95,4.75,3.75,1.73,4.60,3.60,1.77,4.70,3.70,1.75,4.97,3.77,1.79,5.00,3.70,1.70,5.00,3.50,1.75,5.18,3.84,1.87,4.87,3.69,1.76,2.06,1.84,2.07,1.85,2.09,1.90,2.00,1.82,0.75,1.92,2.01,1.91,2.03,1.96,2.04,1.88,1.96
96,I1,29/10/2023,17:00,Inter,Roma,1,0,H,0,0,D,19,3,3,1,14,9,8,0,3,4,0,0,1.62,4.00,5.50,1.60,4.00,5.50,1.63,3.95,5.50,1.65,4.00,5.78,1.60,3.90,5.50,1.57,3.50,5.75,1.70,4.10,5.80,1.64,3.94,5.56,2.03,1.87,2.02,1.88,2.04,1.91,1.98,1.86,-0.75,1.82,2.11,1.83,2.10,1.87,2.12,1.80,2.05,1.53,4.00,6.50,1.55,4.00,6.25,1.55,3.95,6.50,1.54,4.21,6.83,1.53,4.20,6.00,1.50,4.10,6.50,1.62,4.35,7.50,1.54,4.13,6.45,2.00,1.90,1.99,1.92,2.01,2.03,1.91,1.91,-1.00,1.94,1.99,1.95,1.97,2.04,2.04,1.91,1.94
97,I1,29/10/2023,19:45,Napoli,Milan,2,2,D,0,2,A,17,20,4,6,14,10,6,4,4,3,1,0,2.20,3.40,3.30,2.20,3.40,3.20,2.25,3.40,3.25,2.30,3.37,3.38,2.15,3.30,3.40,2.15,3.10,3.25,2.32,3.50,3.50,2.25,3.37,3.31,2.04,1.86,2.05,1.85,2.05,1.91,1.99,1.85,-0.25,1.95,1.98,1.97,1.93,1.98,1.99,1.91,1.93,2.20,3.40,3.30,2.20,3.30,3.25,2.25,3.40,3.20,2.27,3.43,3.39,2.20,3.40,3.20,2.20,3.25,3.40,2.37,3.47,3.40,2.26,3.37,3.28,1.95,1.95,1.97,1.93,2.04,2.03,1.94,1.89,-0.25,1.97,1.96,1.95,1.97,2.03,1.97,1